# The GE Analytics Engineer Case Study - Pneumoconiosis Detection

# 1. Introduction

## 1.1 Background

A leading hospital wishes to develop a screening program for coal miners, in order to facilitate early detection of Pneumoconiosis. The standard detection procedure involves taking a chest x-ray and examining it for abnormalities that indicate the onset of Pneumoconiosis. A typical doctor’s report divides each lung into three zones (upper, middle and lower) and labels them as normal/abnormal. However, due to the lack of trained doctors with expertise and the large number of patients to be screened, they have requested GE to develop a computer-aided detection system.

A team of image analysts have already developed algorithms to segment the lung and divide it into three zones. They have done this for a set of images where the doctor’s labeling for the lung zones is known, and characterized each lung zone in terms of a set of features. Each patient is identified by a unique patient number. The feature data for the various lung zones for each patient, along with the zone label (0=Normal, 1=Abnormal) is given in the attached Excel workbook. The workbook have separate tabs for each lung zone. Let's now import the workbook and see head of each one.


In [1]:
%matplotlib inline

import pandas as pd       #data structures and data analysis tools
import os                 #operating system interfaces

PATH = os.getcwd() + '/input/CollatedPneumoconiosisData-GE Internal.xlsx'       #Linux Path syntax
#PATH = os.getcwd() + '\\CollatedPneumoconiosisData-GE Internal.xlsx'           #Windows path syntax

df_RU = pd.read_excel(io = PATH, sheet_name = 'RightUpper')
df_RM = pd.read_excel(io = PATH, sheet_name = 'RightMiddle')
df_RL = pd.read_excel(io = PATH, sheet_name = 'RightLower')

df_LU = pd.read_excel(io = PATH, sheet_name = 'LeftUpper')
df_LM = pd.read_excel(io = PATH, sheet_name = 'LeftMiddle')
df_LL = pd.read_excel(io = PATH, sheet_name = 'LeftLower')

In [ ]:
df_RU.head()

In [ ]:
df_RM.head()

In [ ]:
df_RL.head()

In [ ]:
df_LU.head()

In [ ]:
df_LM.head()

In [ ]:
df_LL.head()

Data tables looks very simialar. First column is Patient ID, then we have numeric continuous values representing different independent variables and last column is label for dependend variable with patient condition. The independent variables are set of features processed from x-ray images by a team. Study instructions provide detailed description of available features.

## 1.2 Feature description

Each region of interest (lung zone) is characterized in terms of a set of features. Two types of features are extracted in order to describe each region of interest. These are described below:

### a) Intensity based
A set of 6 features are extracted based on the histogram of intensity values – mean, standard deviation, skewness, kurtosis, energy and entropy. Apart from calculating these on the original ROI, these features are also extracted after applying a difference filter on the image for the purpose of local enhancement. If I(x,y) denotes the image gray value at (x,y), the first and second order filters are defined as:

$$L_{1}^{\theta}(d) = f_{x}cos\theta + f_{y}sin\theta$$

$$L_{2}^{\theta}(d) = f_{xx}cos^{2}\theta + f_{yy}sin^{2}\theta + f_{x}cos\theta$$

where: 
- $d$ is the difference scale,
- $\theta$ is the orientation at which the difference is computed,
-$f_{x}$ and $f_{y}$ represent the first order difference,
- $f_{xx},f_{yy},f_{xy}$ represent the second order difference.
 
First and second order difference filter bank are used with given orientations $\theta\in\{0,30,35,60,90,120,135,150,180\}$ and given scale $d\in\{1,2\}$. Six intensity-based features are calculated (mean, variance, skewness, kurtosis, energy, entropy) for each filtered image, along with the same features for the raw image without filtering, amounting to a total of 222 features. A subset of 34 features from this set has been provided in the data sheet. These features are labeled with the prefix $Hist\_d\_θ$.

### b) Co-occurrence matrix based
A set of 5 features are also extracted based on the gray level co-occurrence matrix computed for the ROI, namely energy, entropy, local homogeneity, correlation and inertia. The co-occurrence matrix allows to capture the level of similarity and dissimilarity among adjacent pixels in an ROI. Thus, an ROI with an opacity will contain adjacent pixels with similarly high intensities, whereas a normal ROI will not contain such adjacent pixels. Computing these features for various orientations $\delta=\{0,45,90,135\}$ captures this information for various types of adjacency. A subset of 5 of out of 25 such features has been provided in the attached data sheet. These features are labeled with the prefix $CoMatrix\_Deg\delta$.

## 2. Exploratory analysis

### 2.1 General overview of the data

The input data is stored in different data frame. The first step is to check if we have any empty or missing value and see what are frames shapes.

In [ ]:
df_RU.isnull().values.any()

In [ ]:
df_RM.isnull().values.any()

In [ ]:
df_RL.isnull().values.any()

In [ ]:
df_LU.isnull().values.any()

In [ ]:
df_LM.isnull().values.any()

In [ ]:
df_LL.isnull().values.any()

There are no missing values in the data frames. Next step is to see what are shapes of data frames.

In [ ]:
RU_shape = df_RU.shape
RM_shape = df_RM.shape
RL_shape = df_RL.shape
LU_shape = df_LU.shape
LM_shape = df_LM.shape
LL_shape = df_LL.shape

print("Shape of Right Upper zone table is: ", RU_shape)
print("Shape of Right Middle zone table is: ", RM_shape)
print("Shape of Right Lower zone table is: ", RL_shape)
print("Shape of Left Upper zone table is: ", LU_shape)
print("Shape of Left Middle zone table is: ", LM_shape)
print("Shape of Left Lower zone table is: ", LL_shape)

For every lung zone we have the same number of columns, but the number of rows is different. To examine this let's check if there are any duplicates for patient IDs. We will also plot a histograms to see what is distribution of number of zones examined for each patient.

In [ ]:
ids = df_RU['PatientNumMasked']
print('Right Upper data frame has', 
      len(df_RU[ids.isin(ids[ids.duplicated()])].sort_values('PatientNumMasked')), 
      'duplicates.')

In [ ]:
ids = df_RM['PatientNumMasked']
print('Right Medium data frame has',
      len(df_RM[ids.isin(ids[ids.duplicated()])].sort_values('PatientNumMasked')),
      'duplicates.')

In [ ]:
ids = df_RL['PatientNumMasked']
print('Right Lower data frame has', 
      len(df_RL[ids.isin(ids[ids.duplicated()])].sort_values('PatientNumMasked')), 
      'duplicates.')

In [ ]:
ids = df_LU['PatientNumMasked']
print('Left Upper data frame has',
      len(df_LU[ids.isin(ids[ids.duplicated()])].sort_values('PatientNumMasked')),
      'duplicates.')

In [ ]:
ids = df_LM['PatientNumMasked']
print('Left Medium data frame has',
      len(df_LM[ids.isin(ids[ids.duplicated()])].sort_values('PatientNumMasked')),
      'duplicates.')

In [ ]:
ids = df_LL['PatientNumMasked']
print('Left Lower data frame has',
      len(df_LL[ids.isin(ids[ids.duplicated()])].sort_values('PatientNumMasked')),
      'duplicates.')

There are no duplicates, so there are patients that did not have exam for every lung zone. Histograms below are showing what is the distribution of data for each patient by lung zone

In [ ]:
import numpy as np                       #linear algebra
import matplotlib.pyplot as plt          #plotting library
from matplotlib import gridspec

x1 = np.arange(6)
xt = ('Left Upper', 'Left Medium', 'Left Lowe',
     'Right Upper', 'Right Medium', 'Right Lower')
y1 = [LU_shape[0], LM_shape[0], LL_shape[0],
      RU_shape[0], RM_shape[0], RL_shape[0]]

df_temp = pd.concat([df_RU['PatientNumMasked'], df_RM['PatientNumMasked'], df_RL['PatientNumMasked'], 
                     df_LU['PatientNumMasked'], df_LM['PatientNumMasked'], df_LL['PatientNumMasked']], axis=0)
exam_counts = df_temp.value_counts()


fig = plt.figure(figsize=(16, 5))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 3]) 
ax0 = plt.subplot(gs[0])
ax0.set_xticklabels(xt, fontdict=None, minor=False, fontsize=14)
plt.sca(ax0)
plt.xticks(rotation=45)
plt.yticks(fontsize=14)
ax0.set_title('Number of exams per lung zone', fontsize=20)
ax0.bar(x1, y1, color='cornflowerblue', linewidth=0)
ax1 = plt.subplot(gs[1])
ax1.hist(exam_counts, color='cornflowerblue', linewidth=0)
ax1.set_title('Number of examps per patient', fontsize=20)
plt.sca(ax1)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

del df_temp            #delete objects that will not be used later
del exam_counts

Right and lung zone have similar data distribution. Medium zone of each lung have the most observations, and upper zone have least observations. Majority of patients have full exam of six lung zone.

At this point it is important to remember that Pneumoconiosis is labeled for every zone examined for a patient even if only one zone indicate a disease. This can be a problematic when building a model, since healthy zones are classified in training data as ill, when only one zone is ill.

Important apect of data for building predictive model is balance of data labels. Balanced classes don't need much attantion, while imbalanced classes need to be treated carefully.

In [ ]:
print('Fraction of positive classes in Upper Left: ', len(df_LU[df_LU['Label']==1])/len(df_LU))
print('Fraction of positive classes in Medium Left: ', len(df_LM[df_LM['Label']==1])/len(df_LM))
print('Fraction of positive classes in Lower Left: ', len(df_LL[df_LL['Label']==1])/len(df_LL))
print('Fraction of positive classes in Upper Right: ', len(df_RU[df_RU['Label']==1])/len(df_RU))
print('Fraction of positive classes in Upper Medium: ', len(df_RM[df_RM['Label']==1])/len(df_RM))
print('Fraction of positive classes in Upper Left: ', len(df_RL[df_RL['Label']==1])/len(df_RL))

Different lung zones yelds slightly different result, overall result is about 35% of positive class. Classes are imbalanced, but imbalance is not big. Depend on final scores, this might need to be adressed.

### 2.2 Univariate analysis

Univariate analysis is first step to get better understanding of independent variables. Provided data contain continuous numerical data. Section below will go into the features one by one. Total of 39 features will be analized to detect any outliers and to compare distributions between two dependent variables, central tendency and spread of the independent variable. Different lung zones will be analized separatelly.

In [ ]:
import seaborn as sns

df_RU['Zone'] = 'Right Upper'
df_RM['Zone'] = 'Right Medium'
df_RL['Zone'] = 'Right Lower'
df_LU['Zone'] = 'Left Upper'
df_LM['Zone'] = 'Left Medium'
df_LL['Zone'] = 'Left Lower'


columns_to_exclude = ['PatientNumMasked', 'Label', 'Zone']
for col in df_RU.columns:
    if col in columns_to_exclude:
        pass
    else:
        df = pd.concat([df_RU[[col, 'Zone', 'Label']], 
                        df_RM[[col, 'Zone', 'Label']], 
                        df_RL[[col, 'Zone', 'Label']],
                        df_LU[[col, 'Zone', 'Label']], 
                        df_LM[[col, 'Zone', 'Label']], 
                        df_LL[[col, 'Zone', 'Label']]], axis=0)

        fig = plt.figure(figsize=(16, 5))
        ax = plt.axes()
        plt.hold(True)

        bp = sns.boxplot(x='Zone', y=col, hue='Label', data=df, palette='RdBu_r')
        plt.title('Boxplots of feature '+col+' by lung zones.', fontsize=20)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.axes().get_yaxis().get_label().set_visible(False)
        plt.axes().get_xaxis().get_label().set_visible(False)
        L=plt.legend(fontsize=14, loc=2)
        L.get_texts()[0].set_text('Healthy')
        L.get_texts()[1].set_text('Sick')
        plt.show()
        plt.close()
        

Univariate analysis revealed many outliers in the dataset. Worth to mention is that extreem outliers occurs only in 'Healthy' class. Some of features contains very sevire outliers (ex. Hist_2_180_2_Kurtosis Right Lower zone). These outliers will need to be handeled in later part. During the analysis differences between class labels are visible for some features (ex. Hist_2_135_1_Entropy).

### 2.3 Multivariate analysis

Multivariate is second step, it is used to see relationship between features. Because dataset contains only continuous features, during the analysis we will only compare these type of features with breakdown into class labels to get additional information.

Comparing every feature will result in 39x39 grid which is too much.

In [ ]:
dict_zones = {'Right Upper': df_RU, 'Right Medium': df_RM, 'Righ Lower': df_RL,
             'Left Upper': df_LU, 'Left Medium': df_LM, 'Left Lower': df_LL}
columns_all = df_RU.columns.tolist()         #columns in every dataframe are the same
columns_keep = set(columns_all) - set(columns_to_exclude)

for key, value in dict_zones.items():
    df_hm = value[list(columns_keep)]
    cor = df_hm.corr()               #Pearson correlation coefficients
      
    fig = plt.figure(figsize=(30, 30))
    hm = sns.heatmap(cor, annot=True, fmt=".2f", cmap='RdBu_r')
    plt.title('Heatmap of features for '+key+' lung zone', fontsize=35)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)
    cax = plt.gcf().axes[-1]
    cax.tick_params(labelsize=25)
    plt.show()
    
    cor = df_hm.corr().abs()
    mask = np.zeros_like(cor)
    mask[np.triu_indices_from(mask)] = True
    cor = cor*mask
    np.fill_diagonal(cor.values, -2)
    s = cor.unstack()
    s= s.sort_values(ascending=False)
    df_corr = pd.DataFrame(s, columns=['Correlation'])
    df_corr = df_corr.sort_values(by='Correlation', ascending=False)
    df_corr = df_corr[df_corr['Correlation']>0.7]
    print('Table of features Pearson correlation coefficients (corr > 0.7) for ',key,' lung zone\n')
    print(df_corr)

del dict_zones
del df_hm

###### Would be good to fix range for colorbar between heatmaps #######

## 3. Building model and predictions



### 3.1 

In [2]:
#Patient ids were removed to avoid data leakage

X_col = ['Hist_0_0_0_Mean', 'Hist_0_0_0_Skewness',
       'Hist_0_0_0_Kurtosis', 'Hist_0_0_0_Entropy', 'Hist_2_45_1_Entropy',
       'Hist_2_60_1_Skewness', 'Hist_2_90_1_Skewness', 'Hist_2_90_1_Kurtosis',
       'Hist_2_135_1_Entropy', 'Hist_1_150_1_Skewness',
       'Hist_2_180_1_Skewness', 'Hist_1_30_2_Mean', 'Hist_2_30_2_Mean',
       'Hist_2_30_2_Entropy', 'Hist_2_60_2_Skewness', 'Hist_2_60_2_Kurtosis',
       'Hist_1_90_2_Skewness', 'Hist_2_90_2_Mean', 'Hist_2_90_2_Skewness',
       'Hist_2_90_2_Kurtosis', 'Hist_1_120_2_Mean', 'Hist_1_135_2_Mean',
       'Hist_1_135_2_Entropy', 'Hist_2_150_2_Mean', 'Hist_2_150_2_Skewness',
       'Hist_2_150_2_Kurtosis', 'Hist_2_150_2_Entropy', 'Hist_1_180_2_Mean',
       'Hist_1_180_2_StdDev', 'Hist_1_180_2_Skewness', 'Hist_2_180_2_Mean',
       'Hist_2_180_2_Skewness', 'Hist_2_180_2_Kurtosis',
       'Hist_2_180_2_Entropy', 'CoMatrix_Deg45_Local_Homogeneity',
       'CoMatrix_Deg90_Local_Homogeneity', 'CoMatrix_Deg135_Local_Homogeneity',
       'CoMatrix_Deg135_Correlation', 'CoMatrix_Deg135_Inertia']
y_col = ['Label']

In [3]:
def treat_outliers(df, columns):
    from tqdm import tqdm_notebook
    for col in tqdm_notebook(df[columns].columns, total=len(df[columns].columns)):
        p25, p50, p75 = np.percentile(df[col], 0.25), np.percentile(df[col], 0.5), np.percentile(df[col], 0.75)
        iqr = p75 - p25
        #Labels 1
        df.loc[df.Label == 1, col] = np.where(df[df['Label']==1][col] > p75 + 100*iqr, p50, df[df['Label']==1][col])
        df.loc[df.Label == 1, col] = np.where(df[df['Label']==1][col] < p25 - 100*iqr, p50, df[df['Label']==1][col])
        #Labels0
        df.loc[df.Label == 0, col] = np.where(df[df['Label']==0][col] > p75 + 100*iqr, p50, df[df['Label']==0][col])
        df.loc[df.Label == 0, col] = np.where(df[df['Label']==0][col] < p25 - 100*iqr, p50, df[df['Label']==0][col])        
        
    return df

In [4]:
#Outliers reduction has meaningfull impact on score
import numpy as np
df_RU = treat_outliers(df_RU, X_col)
df_RM = treat_outliers(df_RM, X_col)
df_RL = treat_outliers(df_RL, X_col)

df_LU = treat_outliers(df_LU, X_col)
df_LM = treat_outliers(df_LM, X_col)
df_LL = treat_outliers(df_LL, X_col)

HBox(children=(IntProgress(value=0, max=39), HTML(value='')))

HBox(children=(IntProgress(value=0, max=39), HTML(value='')))

HBox(children=(IntProgress(value=0, max=39), HTML(value='')))

HBox(children=(IntProgress(value=0, max=39), HTML(value='')))

HBox(children=(IntProgress(value=0, max=39), HTML(value='')))

HBox(children=(IntProgress(value=0, max=39), HTML(value='')))

In [5]:
X_RU = df_RU[X_col]
X_RM = df_RM[X_col]
X_RL = df_RL[X_col]
X_LU = df_LU[X_col]
X_LM = df_LM[X_col]
X_LL = df_LL[X_col]

y_RU = df_RU[y_col]
y_RM = df_RM[y_col]
y_RL = df_RL[y_col]
y_LU = df_LU[y_col]
y_LM = df_LM[y_col]
y_LL = df_LL[y_col]

RU_ids = df_RU['PatientNumMasked'].values
RM_ids = df_RM['PatientNumMasked'].values
RL_ids = df_RL['PatientNumMasked'].values
LU_ids = df_LU['PatientNumMasked'].values
LM_ids = df_LM['PatientNumMasked'].values
LL_ids = df_LL['PatientNumMasked'].values

In [17]:
import sklearn
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.cross_validation import LeaveOneOut
from sklearn.metrics import recall_score
from xgboost import XGBClassifier

import xgboost
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#X_train, X_test, y_train, y_test = train_test_split(
#    df_LL[X_col], df_LL[y_col], test_size=0.33, random_state=42)

#grid_values = {'n_jobs':[4],
#              'objective':['binary:logistic'],
#              'learning_rate': np.arange(0.01, 0.1, 0.01),
#               'max_depth': np.arange(1, 6, 1),
#               'n_estimators': np.arange(100, 1000, 100),
#               'gamma': np.arange(0, 0.05, 0.01),
#               'max_delta_step': np.arange(0, 0.05, 0.01),
#               'reg_alpha': np.arange(0, 0.5, 0.1),
#               'reg_lambda': np.arange(0.5, 1, 0.1),
#               'scale_pos_weight': np.arange(0.7, 1.3, 0.1),
#               'random_state': [8]

#grid_values = {'n_jobs': [4],
#              'objective':['binary:logistic'],
#              'learning_rate': np.arange(0.01, 0.1, 0.01),
#               'max_depth': np.arange(1, 6, 1),              
#              }

grid_values = {}
              
loo = LeaveOneOut(len(df_LL[y_col]))
111
clf_xgb = XGBClassifier()
grid_clf_recall = GridSearchCV(clf_xgb, grid_values, scoring='recall',cv=loo)
grid_clf_recall.fit(df_LL[X_col], df_LL[y_col])

#y_xgb_recall = grid_clf_recall.decision_function(X_test)
#print('Best Recall: ', recall_score(y_test, y_xgb_recall))
#print('Best parameters: ', grid_clf_recall.best_params_)

GridSearchCV(cv=sklearn.cross_validation.LeaveOneOut(n=434),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='recall', verbose=0)

In [20]:
best_parameters, score, _ = max(grid_clf_recall.grid_scores_, key=lambda x: x[1])
print(best_parameters)
print(score)
#{'n_estimators': 600, 'gamma': 0, 'max_depth': 2, 
#'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.07, 'max_delta_step': 0.0}
#{'max_depth': 4, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.05}

{}
0.29723502304147464


In [15]:
grid_clf_recall.grid_scores_

[mean: 0.22811, std: 0.41961, params: {'max_depth': 1, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.01},
 mean: 0.25115, std: 0.43368, params: {'max_depth': 2, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.01},
 mean: 0.25346, std: 0.43499, params: {'max_depth': 3, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.01},
 mean: 0.27419, std: 0.44611, params: {'max_depth': 4, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.01},
 mean: 0.26959, std: 0.44374, params: {'max_depth': 5, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.01},
 mean: 0.24194, std: 0.42826, params: {'max_depth': 1, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.02},
 mean: 0.26498, std: 0.44132, params: {'max_depth': 2, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.02},
 mean: 0.27189, std: 0.44493, params: {'max_depth': 3, 'n_jobs': 4, 'objective': 'binary:logistic', 'learning_rate': 0.02},
 mean: 0

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm_notebook

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

loo = LeaveOneOut()
clf_rf = RandomForestClassifier(n_estimators=10, criterion='gini', min_samples_split=2, 
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                min_impurity_decrease=0.0, bootstrap=True)

predictions_rf = []

for train, test in tqdm_notebook(loo.split(X), total=len(y)):
    one = y.loc[test].index
    rest = y.loc[train].index

    X_test_loo = X.loc[one]
    X_train_loo = X.loc[rest]
    y_test_loo = y.loc[one]
    y_train_loo = y.loc[rest]
    
    clf_rf.fit(X_train_loo, y_train_loo.values.ravel())
    predictions_rf.append(clf_rf.predict(X_test_loo)[0])
    
predictions_rf = np.array(predictions_rf).reshape(len(y),1)    
    
print('Precission score: ', precision_score(y, predictions_rf))
print('Accuracy score: ', accuracy_score(y, predictions_rf))
print('Recall score: ', recall_score(y, predictions_rf))

#Default
#Precission score:  0.9108910891089109
#Accuracy score:  0.9183673469387755
#Recall score:  0.8

In [ ]:
def xgb_predictions(X, y):
    import xgboost
    from sklearn.model_selection import LeaveOneOut
    from tqdm import tqdm_notebook
    import warnings
    warnings.filterwarnings("ignore")
    
    loo = LeaveOneOut()

    clf_xgb = xgboost.XGBClassifier(learning_rate =0.1,
                                     n_estimators=500,
                                     max_depth=5,
                                     min_child_weight=1,
                                     gamma=0,
                                     subsample=0.8,
                                     colsample_bytree=0.8,
                                     objective= 'binary:logistic',
                                     nthread=4,
                                     scale_pos_weight=1,
                                     seed=27)

    predictions_xgb = []

    for train, test in tqdm_notebook(loo.split(X), total=len(y)):
        one = y.loc[test].index
        rest = y.loc[train].index

        X_test_loo = X.loc[one]
        X_train_loo = X.loc[rest]
        y_test_loo = y.loc[one]
        y_train_loo = y.loc[rest]
        clf_xgb.fit(X_train_loo, y_train_loo.values.ravel())
        predictions_xgb.append(clf_xgb.predict(X_test_loo)[0])
    
    return np.array(predictions_xgb).reshape(len(y),1)

In [ ]:
predictions_RU = xgb_predictions(X_RU, y_RU).reshape((len(y_RU),))
predictions_RM = xgb_predictions(X_RM, y_RM).reshape((len(y_RM),))
predictions_RL = xgb_predictions(X_RL, y_RL).reshape((len(y_RL),))
predictions_LR = xgb_predictions(X_LU, y_LU).reshape((len(y_LU),))
predictions_LM = xgb_predictions(X_LM, y_LM).reshape((len(y_LM),))
predictions_LL = xgb_predictions(X_LL, y_LL).reshape((len(y_LL),))

In [ ]:
predictions_RU = predictions_RU.reshape((len(y_RU),))
predictions_RM = predictions_RM.reshape((len(y_RM),))
predictions_RL = predictions_RL.reshape((len(y_RL),))
predictions_LU = predictions_LR.reshape((len(y_LU),))
predictions_LM = predictions_LM.reshape((len(y_LM),))
predictions_LL = predictions_LL.reshape((len(y_LL),))

y_RU = y_RU.values.reshape((len(y_RU),))
y_RM = y_RM.values.reshape((len(y_RM),))
y_RL = y_RL.values.reshape((len(y_RL),))
y_LU = y_LU.values.reshape((len(y_LU),))
y_LM = y_LM.values.reshape((len(y_LM),))
y_LL = y_LL.values.reshape((len(y_LL),))

In [ ]:
df_RU_pred = pd.DataFrame({'y_pred_RU': predictions_RU, 'y_true_RU': y_RU}, index=RU_ids)
df_RM_pred = pd.DataFrame({'y_pred_RM': predictions_RM, 'y_true_RM': y_RM}, index=RM_ids)
df_RL_pred = pd.DataFrame({'y_pred_RL': predictions_RL, 'y_true_RL': y_RL}, index=RL_ids)
df_LU_pred = pd.DataFrame({'y_pred_LU': predictions_LU, 'y_true_LU': y_LU}, index=LU_ids)
df_LM_pred = pd.DataFrame({'y_pred_LM': predictions_LM, 'y_true_LM': y_LM}, index=LM_ids)
df_LL_pred = pd.DataFrame({'y_pred_LL': predictions_LL, 'y_true_LL': y_LL}, index=LL_ids)

In [ ]:
df_ensemble = df_RU_pred.merge(df_RM_pred, how='outer', right_index=True, left_index=True)
df_ensemble = df_ensemble.merge(df_RL_pred, how='outer', right_index=True, left_index=True)
df_ensemble = df_ensemble.merge(df_LU_pred, how='outer', right_index=True, left_index=True)
df_ensemble = df_ensemble.merge(df_LM_pred, how='outer', right_index=True, left_index=True)
df_ensemble = df_ensemble.merge(df_LL_pred, how='outer', right_index=True, left_index=True)
df_ensemble['y_pred'] = np.sum(df_ensemble[['y_pred_RU','y_pred_RM','y_pred_RL',
                                        'y_pred_LU', 'y_pred_LM', 'y_pred_LL']], axis=1)
df_ensemble['y_true'] = np.sum(df_ensemble[['y_true_RU','y_true_RM','y_true_RL',
                                        'y_true_LU', 'y_true_LM', 'y_true_LL']], axis=1)

df_ensemble.loc[df_ensemble['y_pred'] > 0, 'y_pred'] = 1
df_ensemble.loc[df_ensemble['y_true'] > 0, 'y_true'] = 1
df_ensemble[(df_ensemble['y_true']==1) & (df_ensemble['y_pred']==0)]

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

print('Precission score: ',precision_score(df_ensemble['y_true'], df_ensemble['y_pred']))
print('Accuracy score: ', accuracy_score(df_ensemble['y_true'], df_ensemble['y_pred']))
print('Recall score: ', recall_score(df_ensemble['y_true'], df_ensemble['y_pred']))

#First shot
#Precission score:  0.8
#Accuracy score:  0.8710359408033826
#Recall score:  0.9183673469387755



ideas: 
- heatmap: add table with feature pairs that corr>0.7
- Dimmensionality reduction: https://en.wikipedia.org/wiki/Principal_component_analysis 




Sections:
- Introduction
- methods
- results
- conclusions

include discussion of the data used, exploratory analysis, model type(s) identification, estimation, diagnosis, validation, and recomendations. 